In [2]:
# ============================================================
# Koltsov3 perm_type=2 report (single HTML)
# Works with CSV columns:
# k_param,perm_type,coset,n_values,diameters,last_layers,total_states,num_n_computed
# ============================================================

import os, json, ast
import pandas as pd
import numpy as np
import plotly.express as px

# ----------------------------
# Config
# ----------------------------
CSV_PATH = "results/koltsov3_4different_perm2_results.csv"         # <-- change to your actual filename
OUT_HTML = "results/koltsov3_perm2_report.html" # output report

# Keep plots readable
N_MIN = 20        # show only n >= this
N_MAX = 40        # show only n <= this
HEATMAP_NS = [24, 30, 36, 40]  # snapshots
DEFAULT_KS = 6    # number of k lines shown by default (others hidden but selectable)

os.makedirs(os.path.dirname(OUT_HTML), exist_ok=True)

# ----------------------------
# Helpers
# ----------------------------
def parse_listish(x):
    """Parse strings like "[1,2,3]" into python list."""
    if pd.isna(x):
        return []
    if isinstance(x, list):
        return x
    s = str(x).strip()
    try:
        return json.loads(s)
    except Exception:
        try:
            return ast.literal_eval(s)
        except Exception:
            return []

def parse_dictish(x):
    """
    Parse strings like {"4": 10, "5": 12} or {""4"": 10, ...} into dict[int -> int].
    """
    if pd.isna(x):
        return {}
    if isinstance(x, dict):
        # normalize keys
        out = {}
        for k, v in x.items():
            try:
                out[int(k)] = int(v)
            except Exception:
                pass
        return out

    s = str(x).strip()
    # Fix common CSV-escaped JSON: {""4"": 10} -> {"4": 10}
    s = s.replace('""', '"')
    try:
        d = json.loads(s)
    except Exception:
        try:
            d = ast.literal_eval(s)
        except Exception:
            return {}

    out = {}
    for k, v in d.items():
        try:
            out[int(k)] = int(v)
        except Exception:
            continue
    return out

def add_bottom_xlabel(fig, label="n"):
    fig.for_each_xaxis(lambda ax: ax.update(title_text=label))
    fig.update_layout(margin=dict(b=90))
    return fig

# ----------------------------
# Load + parse CSV
# ----------------------------
df = pd.read_csv(CSV_PATH)

# Force types we expect
df["k_param"] = df["k_param"].astype(int)
df["perm_type"] = df["perm_type"].astype(int)
df["coset"] = df["coset"].astype(str)

# Parse columns that are embedded structures
df["n_list"] = df["n_values"].apply(parse_listish)
df["diam_dict"] = df["diameters"].apply(parse_dictish)
df["ll_dict"] = df["last_layers"].apply(parse_dictish)
df["states_dict"] = df["total_states"].apply(parse_dictish)

# ----------------------------
# Expand to long format: one row per (k, n)
# ----------------------------
rows = []
for _, r in df.iterrows():
    k = int(r["k_param"])
    perm = int(r["perm_type"])
    coset = r["coset"]
    for n in r["n_list"]:
        n = int(n)
        diam = r["diam_dict"].get(n, np.nan)
        ll = r["ll_dict"].get(n, np.nan)
        st = r["states_dict"].get(n, np.nan)
        rows.append({
            "k": k,
            "perm_type": perm,
            "coset": coset,
            "n": n,
            "diameter": diam,
            "last_layer_size": ll,
            "total_states": st
        })

long_df = pd.DataFrame(rows).dropna(subset=["diameter"])
long_df["diameter"] = long_df["diameter"].astype(int)
long_df["n_mod_2"] = (long_df["n"] % 2).astype(int).astype(str)
long_df["diam_over_n2"] = long_df["diameter"] / (long_df["n"] ** 2)

# Apply n-window to reduce clutter
long_df_window = long_df[(long_df["n"] >= N_MIN) & (long_df["n"] <= N_MAX)].copy()

# Convenience filtered frames
ll_df_window = long_df_window.dropna(subset=["last_layer_size"]).copy()
states_df_window = long_df_window.dropna(subset=["total_states"]).copy()

# ----------------------------
# Heatmap helper (k vs n) at a fixed n0
# ----------------------------
def heatmap_at_n(n0):
    sub = long_df[long_df["n"] == n0].copy()
    if sub.empty:
        return None
    # Pivot with coset as a separate facet column if multiple cosets exist
    sub["coset"] = sub["coset"].astype(str)
    fig = px.density_heatmap(
        sub,
        x="k",
        y="coset",
        z="diameter",
        histfunc="avg",
        title=f"Heatmap snapshot at n={n0}: avg diameter (y=coset, x=k)"
    )
    fig.update_layout(height=max(300, 60 * sub["coset"].nunique()))
    fig.update_xaxes(title_text="k")
    fig.update_yaxes(title_text="coset")
    return fig

# ----------------------------
# Create figures
# ----------------------------
figs = []

# 1) Heatmaps for chosen n values
for n0 in HEATMAP_NS:
    fig = heatmap_at_n(n0)
    if fig is not None:
        figs.append(("Heatmap", fig))

# 2) Diameter vs n (colored by k) + k selector (legend + dropdown)
all_ks = sorted(long_df_window["k"].unique())
default_ks = all_ks[:min(DEFAULT_KS, len(all_ks))]

fig_diam_vs_n = px.line(
    long_df_window.sort_values(["k", "n"]),
    x="n",
    y="diameter",
    color="k",
    line_group="k",
    title=f"perm_type=2 — Diameter vs n (n in [{N_MIN},{N_MAX}]) — legend click hides/shows k"
)
fig_diam_vs_n.update_xaxes(range=[N_MIN, N_MAX])
add_bottom_xlabel(fig_diam_vs_n, "n")

# Hide most ks initially (still selectable)
for tr in fig_diam_vs_n.data:
    try:
        k_val = int(tr.name)
    except Exception:
        continue
    if k_val not in default_ks:
        tr.visible = "legendonly"

def vis_mask_for_ks(ks_to_show):
    mask = []
    for tr in fig_diam_vs_n.data:
        try:
            k_val = int(tr.name)
            mask.append(True if k_val in ks_to_show else "legendonly")
        except Exception:
            mask.append(True)
    return mask

even_ks = [k for k in all_ks if k % 2 == 0]
odd_ks  = [k for k in all_ks if k % 2 == 1]

buttons = [
    dict(label=f"Default (first {len(default_ks)})", method="update",
         args=[{"visible": vis_mask_for_ks(default_ks)}]),
    dict(label="Show ALL k", method="update",
         args=[{"visible": [True] * len(fig_diam_vs_n.data)}]),
    dict(label="Hide ALL k", method="update",
         args=[{"visible": ["legendonly"] * len(fig_diam_vs_n.data)}]),
    dict(label="Even k only", method="update",
         args=[{"visible": vis_mask_for_ks(even_ks)}]),
    dict(label="Odd k only", method="update",
         args=[{"visible": vis_mask_for_ks(odd_ks)}]),
]
fig_diam_vs_n.update_layout(
    height=520,
    margin=dict(l=50, r=20, t=80, b=90),
    updatemenus=[dict(
        type="dropdown",
        direction="down",
        x=1.02, xanchor="left",
        y=1.0, yanchor="top",
        buttons=buttons
    )],
    legend_title_text="k (click to toggle)"
)
figs.append(("Diameter vs n", fig_diam_vs_n))

# 3) Parity scatter: diameter vs n colored by n mod 2
fig_parity = px.scatter(
    long_df_window,
    x="n",
    y="diameter",
    color="n_mod_2",
    symbol="n_mod_2",
    hover_data=["k", "coset"],
    title=f"perm_type=2 — Parity test: diameter vs n (n in [{N_MIN},{N_MAX}])"
)
fig_parity.update_layout(height=520, margin=dict(l=50, r=20, t=60, b=90))
add_bottom_xlabel(fig_parity, "n")
figs.append(("Parity", fig_parity))

# 4) Last-layer size vs diameter
if not ll_df_window.empty:
    fig_ll = px.scatter(
        ll_df_window,
        x="diameter",
        y="last_layer_size",
        color="k",
        hover_data=["n", "coset"],
        title=f"perm_type=2 — Last-layer size vs diameter (n in [{N_MIN},{N_MAX}])"
    )
    fig_ll.update_layout(height=520, margin=dict(l=50, r=20, t=60, b=80))
    fig_ll.update_xaxes(title_text="diameter")
    fig_ll.update_yaxes(title_text="last_layer_size")
    figs.append(("Last layer", fig_ll))

# 5) Normalized diameter vs k at max n (within available data)
n_max = int(long_df["n"].max())
sub_norm = long_df[long_df["n"] == n_max].copy()
if not sub_norm.empty:
    fig_norm = px.scatter(
        sub_norm,
        x="k",
        y="diam_over_n2",
        hover_data=["coset", "n"],
        title=f"perm_type=2 — Normalized diameter at n={n_max}: diameter / n² vs k"
    )
    fig_norm.update_layout(height=520, margin=dict(l=50, r=20, t=60, b=80))
    fig_norm.update_xaxes(title_text="k")
    fig_norm.update_yaxes(title_text="diameter / n²")
    figs.append(("Normalized", fig_norm))

# 6) Total states vs n (log scale) — beware: huge integers, plotly handles as float
if not states_df_window.empty:
    fig_states = px.line(
        states_df_window.sort_values(["k", "n"]),
        x="n",
        y="total_states",
        color="k",
        line_group="k",
        log_y=True,
        title=f"perm_type=2 — Total states vs n (log y, n in [{N_MIN},{N_MAX}])"
    )
    fig_states.update_layout(height=520, margin=dict(l=50, r=20, t=60, b=90))
    add_bottom_xlabel(fig_states, "n")
    figs.append(("Total states", fig_states))

# ----------------------------
# Write single HTML report
# ----------------------------
divs = []
for i, (name, fig) in enumerate(figs):
    div = fig.to_html(full_html=False, include_plotlyjs=("cdn" if i == 0 else False))
    divs.append(f"<h2 style='font-family: sans-serif;'>{name}</h2>\n{div}")

summary = f"""
<ul style="font-family: sans-serif; line-height: 1.5;">
  <li><b>Source CSV:</b> {CSV_PATH}</li>
  <li><b>Rows (k parameter sets):</b> {len(df)}</li>
  <li><b>Points (k,n) total:</b> {len(long_df)}</li>
  <li><b>perm_type:</b> {sorted(df['perm_type'].unique())}</li>
  <li><b>k range:</b> {min(df['k_param'])} .. {max(df['k_param'])}</li>
  <li><b>coset(s):</b> {", ".join(sorted(df['coset'].astype(str).unique()))}</li>
  <li><b>Displayed n window:</b> [{N_MIN}, {N_MAX}]</li>
  <li><b>Heatmap snapshots at n:</b> {HEATMAP_NS}</li>
</ul>
"""

html = f"""
<!doctype html>
<html>
<head>
  <meta charset="utf-8"/>
  <title>Koltsov3 perm2 Report</title>
</head>
<body style="margin: 24px;">
  <h1 style="font-family: sans-serif;">Koltsov3 Report — perm_type=2</h1>
  {summary}
  {'<hr/>'.join(divs)}
</body>
</html>
"""

with open(OUT_HTML, "w", encoding="utf-8") as f:
    f.write(html)

print("Wrote report:", OUT_HTML)


Wrote report: results/koltsov3_perm2_report.html
